In [1]:
import sqlite3


In [2]:
conn = sqlite3.connect('../../preproc/tmp/preproc.sqlite3')

In [3]:
c = conn.cursor()
c.execute("select * from task")

In [4]:
columns = [c for c, *_ in c.description]
columns

['id',
 'name',
 'status',
 'meta',
 'type',
 'error',
 'creation_date',
 'submission_date',
 'run_date',
 'done_date']

In [5]:
data = [row for row in c.fetchall()]

In [6]:
values = [dict(zip(columns, row)) for row in data]

In [ ]:
values[:2]

In [7]:
import pandas as pd
import numpy as np
import json
data = (
    pd.DataFrame(values)
    .assign(meta=lambda d: d.meta.map(json.loads))
    .assign(samples=lambda d: d.meta.map(lambda m: m.get('samples', [])))
    .assign(platform=lambda d: d.meta.map(lambda m: m.get('platform', np.nan)))
)
data[:10]

,creation_date,done_date,error,id,meta,name,run_date,status,submission_date,type,samples,platform
0,2015-08-21 18:46:27.696759,None,None,1,"{'platform': 'GPL570', 'accession': 'GSE53224'...",GSE53224_GPL570,None,done,None,preproc,"[GSM1287918, GSM1287919, GSM1287920, GSM128792...",GPL570
1,2015-08-21 18:46:27.698587,None,None,2,"{'platform': 'GPL570', 'accession': 'GSE53183'...",GSE53183_GPL570,None,done,None,preproc,"[GSM1286097, GSM1286098, GSM1286099, GSM128610...",GPL570
2,2015-08-21 18:46:27.699732,None,None,3,"{'platform': 'GPL570', 'accession': 'GSE53157'...",GSE53157_GPL570,None,done,None,preproc,"[GSM1283115, GSM1283116, GSM1283117, GSM128311...",GPL570
3,2015-08-21 18:46:27.700555,None,None,4,"{'platform': 'GPL5082', 'accession': 'GSE65721...",GSE65721_GPL5082,None,skip,None,preproc,"[GSM1603234, GSM1603235, GSM1603236, GSM160323...",GPL5082
4,2015-08-21 18:46:27.701362,None,None,5,"{'platform': 'GPL570', 'accession': 'GSE65721'...",GSE65721_GPL570,None,done,None,preproc,"[GSM1603354, GSM1603355, GSM1603356, GSM160335...",GPL570
5,2015-08-21 18:46:27.702123,None,None,6,"{'platform': 'GPL570', 'accession': 'GSE53092'...",GSE53092_GPL570,None,done,None,preproc,"[GSM1282316, GSM1282317, GSM1282318, GSM1282319]",GPL570
6,2015-08-21 18:46:27.702899,None,None,7,"{'platform': 'GPL570', 'accession': 'GSE65707'...",GSE65707_GPL570,None,done,None,preproc,"[GSM1603354, GSM1603355, GSM1603356, GSM160335...",GPL570
7,2015-08-21 18:46:27.703622,None,None,8,"{'platform': 'GPL570', 'accession': 'GSE53059'...",GSE53059_GPL570,None,done,None,preproc,"[GSM1281432, GSM1281433, GSM1281434, GSM128143...",GPL570
8,2015-08-21 18:46:27.704425,None,None,9,"{'platform': 'GPL570', 'accession': 'GSE53046'...",GSE53046_GPL570,None,done,None,preproc,"[GSM1281025, GSM1281026, GSM1281027, GSM128102...",GPL570
9,2015-08-21 18:46:27.705136,None,None,10,"{'platform': 'GPL570', 'accession': 'GSE53012'...",GSE53012_GPL570,None,done,None,preproc,"[GSM1280329, GSM1280330, GSM1280331, GSM128033...",GPL570


In [ ]:
import seaborn
%matplotlib inline
(data
 .query("platform == 'GPL570' and type == 'preproc' and status == 'done'")
 .samples.map(len)
 .value_counts()
)

In [ ]:
data.query("platform == 'GPL570'").status.value_counts()[:10]

In [ ]:
data.query("platform=='GPL97'")['status'].value_counts()

In [ ]:
data.platform.value_counts().plot(kind='pie')

In [ ]:
(data
 .query("platform == 'GPL1261' and type == 'preproc' and status == 'done'")
 .name.to_csv(path='/Users/nikita/mnt/ls2/preproc/tmp/roshe-genes-corr/to_preroc-1261.txt', index=False)
)

In [ ]:
?pd.read_table

In [ ]:
!mkdir -p /Users/nikita/mnt/ls2/preproc/tmp/roshe-genes-corr/

In [ ]:
drug = pd.read_table('/Users/nikita/Downloads/Archive/mcf7_0.04_mean.txt', skiprows=2, header=None, index_col=0)[1]
drug[:10]

In [ ]:
sample = pd.read_table('../data/roshe-genes-corr/GSM1419638_YX_U133Plus_2_44T.expr', sep=' ', index_col=0)
sample = sample[sample.columns[0]]
sample[:10]

In [ ]:
drug.shape, sample.shape

In [ ]:
import scipy
import numpy as np

In [ ]:
drug.hist(bins=30)

In [ ]:
sample = sample.map(np.log2)

In [ ]:
sample.hist(bins=30)

In [ ]:
d = pd.DataFrame(dict(sample=sample, drug=drug)).dropna()
d.shape

In [ ]:
d.corr()

In [ ]:
sample.corr(drug)

In [ ]:
scipy.stats.pearsonr(d.drug.values, d['sample'].values)

In [ ]:
pd.read_csv('../data/roshe-genes-corr/out/GSE27716_GPL570.csv', index_col=0)

In [ ]:
from glob import glob
from os.path import basename
datasets = [pd.read_csv(f, index_col=0).assign(series=basename(f)) for f in glob('../data/roshe-genes-corr/out/*.csv')]

In [ ]:
corrs = (pd.concat(datasets)
         .reset_index()
         .dropna()
         .assign(platform=lambda d: d.series
                                     .str.split('_')
                                     .str.get(1)
                                     .str.replace('.csv', ''))
         .assign(series=lambda d: d.series
                                     .str.split('_')
                                     .str.get(0))
        )

In [ ]:
(corrs
 .sort(columns=['corr'], ascending=False)
 .drop('index', 1) 
#  .to_csv('../data/roshe-genes-corr/roshe-corrs-GPL570.csv', index=False)
)

In [ ]:
corrs.platform.value_counts()

In [ ]:
def cell():
    done = data.query("type == 'preproc' and status == 'done'").assign(samples=lambda d: d['samples'].map(len))
    t = done.groupby('platform')['samples'].sum()
    t.sort(ascending=False)
    return t

cell()

In [ ]:
data['type'].value_counts()

In [ ]:
data.query("type == 'convert-merge'").apply(lambda x: x['meta']['task'].split('_')[1], axis=1).value_counts()

In [ ]:
data.query("type == 'convert-merge'").status.value_counts()

In [ ]:
data.query("type == 'preproc' and status =='done'").platform.value_counts()[:10]

In [ ]:
data.query("type == 'convert-merge'").meta.map(lambda x: x['task']).to_csv('/Users/nikita/mnt/ls2/preproc/tmp/roshe-genes-corr/to_preroc-genes-570-96-571.txt', index=False)

In [ ]:
data.query("type == 'convert-merge' and status=='done' and platform == 'GPL570'")

In [8]:
converted_tasks = (
    pd.DataFrame(data
     .query("type == 'convert-merge' and status=='done' ")
     .apply(lambda x: x['meta']['task'], axis=1),
    columns=['task']             
    )
    .assign(platform=lambda d: d['task'].map(lambda t: t.split('_')[1]))
    .assign(series=lambda d: d['task'].map(lambda t: t.split('_')[0]))
#     .query("platform == 'GPL570'")
)

converted_tasks

,task,platform,series
13842,GSE65835_GPL570,GPL570,GSE65835
13843,GSE53224_GPL570,GPL570,GSE53224
13844,GSE53183_GPL570,GPL570,GSE53183
13845,GSE53157_GPL570,GPL570,GSE53157
13846,GSE65721_GPL570,GPL570,GSE65721
13847,GSE53092_GPL570,GPL570,GSE53092
13848,GSE65707_GPL570,GPL570,GSE65707
13849,GSE53059_GPL570,GPL570,GSE53059
13850,GSE53046_GPL570,GPL570,GSE53046
13851,GSE53012_GPL570,GPL570,GSE53012


In [10]:
converted_tasks.platform.value_counts()

GPL570    3672
GPL96      693
GPL571     436
dtype: int64

In [ ]:
converted_tasks[['series', 'platform']].to_csv('../data/preproc/converted_tasks.csv', index=False)